In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Irritable_bowel_syndrome_(IBS)"
cohort = "GSE138297"

# Input paths
in_trait_dir = "../../input/GEO/Irritable_bowel_syndrome_(IBS)"
in_cohort_dir = "../../input/GEO/Irritable_bowel_syndrome_(IBS)/GSE138297"

# Output paths
out_data_file = "../../output/preprocess/Irritable_bowel_syndrome_(IBS)/GSE138297.csv"
out_gene_data_file = "../../output/preprocess/Irritable_bowel_syndrome_(IBS)/gene_data/GSE138297.csv"
out_clinical_data_file = "../../output/preprocess/Irritable_bowel_syndrome_(IBS)/clinical_data/GSE138297.csv"
json_path = "../../output/preprocess/Irritable_bowel_syndrome_(IBS)/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"The host response of IBS patients to allogenic and autologous faecal microbiota transfer"
!Series_summary	"In this randomised placebo-controlled trial, irritable bowel syndrome (IBS) patients were treated with faecal material from a healthy donor (n=8, allogenic FMT) or with their own faecal microbiota (n=8, autologous FMT). The faecal transplant was administered by whole colonoscopy into the caecum (30 g of stool in 150 ml sterile saline). Two weeks before the FMT (baseline) as well as two and eight weeks after the FMT, the participants underwent a sigmoidoscopy, and biopsies were collected at a standardised location (20-25 cm from the anal verge at the crossing with the arteria iliaca communis) from an uncleansed sigmoid. In patients treated with allogenic FMT, predominantly immune response-related genes sets were induced, with the strongest response two weeks after FMT. In patients treated with autologous FMT, predominantly metabolism-related g

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# Evaluate if gene expression data is available
is_gene_available = True  # Based on the series summary mentioning gene sets and microarray analysis

# Define data conversion functions and row identifiers
# 1. For trait (IBS)
# According to the background info, all subjects have IBS, but we can distinguish by treatment type
trait_row = 6  # 'experimental condition' in the sample characteristics
def convert_trait(value):
    if value is None:
        return None
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    if 'Allogenic FMT' in value:
        return 1  # Allogenic FMT (treatment group)
    elif 'Autologous FMT' in value:
        return 0  # Autologous FMT (control group)
    return None

# 2. For age
age_row = 3  # 'age (yrs)' in the sample characteristics
def convert_age(value):
    if value is None:
        return None
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    try:
        return float(value)  # Convert to numeric value
    except (ValueError, TypeError):
        return None

# 3. For gender
gender_row = 1  # 'sex (female=1, male=0)' in the sample characteristics
def convert_gender(value):
    if value is None:
        return None
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    if value == '1':
        return 0  # Female (note: we convert to our standard where female=0, male=1)
    elif value == '0':
        return 1  # Male
    return None

# Perform initial filtering and save metadata
is_trait_available = trait_row is not None
initial_validation = validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# If clinical data is available, extract features and save to file
if trait_row is not None:
    # Create a dataframe from the sample characteristics dictionary
    sample_chars = {0: ['tissue: uncleansed colon'], 
                   1: ['sex (female=1, male=0): 1', 'sex (female=1, male=0): 0'], 
                   2: ['subjectid: 1', 'subjectid: 2', 'subjectid: 3', 'subjectid: 4', 'subjectid: 5', 
                       'subjectid: 6', 'subjectid: 7', 'subjectid: 8', 'subjectid: 10', 'subjectid: 11', 
                       'subjectid: 12', 'subjectid: 13', 'subjectid: 14', 'subjectid: 15', 'subjectid: 16'], 
                   3: ['age (yrs): 49', 'age (yrs): 21', 'age (yrs): 31', 'age (yrs): 59', 'age (yrs): 40', 
                       'age (yrs): 33', 'age (yrs): 28', 'age (yrs): 36', 'age (yrs): 50', 'age (yrs): 27', 
                       'age (yrs): 23', 'age (yrs): 32', 'age (yrs): 38'], 
                   4: ['moment of sampling (pre/post intervention): Baseline (app. 2w before intervention)', 
                       'moment of sampling (pre/post intervention): 2 weeks after intervention', 
                       'moment of sampling (pre/post intervention): 8 weeks after intervention'], 
                   5: ['time of sampling: Morning, after overnight fasting'], 
                   6: ['experimental condition: Allogenic FMT', 'experimental condition: Autologous FMT']}
    
    clinical_data = pd.DataFrame.from_dict(sample_chars, orient='index')
    
    # Extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the data
    print("Preview of extracted clinical data:")
    print(preview_df(selected_clinical_df))
    
    # Save to output file
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")


Preview of extracted clinical data:
{0: [1.0, 49.0, 0.0], 1: [0.0, 21.0, 1.0], 2: [nan, 31.0, nan], 3: [nan, 59.0, nan], 4: [nan, 40.0, nan], 5: [nan, 33.0, nan], 6: [nan, 28.0, nan], 7: [nan, 36.0, nan], 8: [nan, 50.0, nan], 9: [nan, 27.0, nan], 10: [nan, 23.0, nan], 11: [nan, 32.0, nan], 12: [nan, 38.0, nan], 13: [nan, nan, nan], 14: [nan, nan, nan]}
Clinical data saved to ../../output/preprocess/Irritable_bowel_syndrome_(IBS)/clinical_data/GSE138297.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. First, let's examine the structure of the matrix file to understand its format
import gzip

# Peek at the first few lines of the file to understand its structure
with gzip.open(matrix_file, 'rt') as file:
    # Read first 100 lines to find the header structure
    for i, line in enumerate(file):
        if '!series_matrix_table_begin' in line:
            print(f"Found data marker at line {i}")
            # Read the next line which should be the header
            header_line = next(file)
            print(f"Header line: {header_line.strip()}")
            # And the first data line
            first_data_line = next(file)
            print(f"First data line: {first_data_line.strip()}")
            break
        if i > 100:  # Limit search to first 100 lines
            print("Matrix table marker not found in first 100 lines")
            break

# 3. Now try to get the genetic data with better error handling
try:
    gene_data = get_genetic_data(matrix_file)
    print(gene_data.index[:20])
except KeyError as e:
    print(f"KeyError: {e}")
    
    # Alternative approach: manually extract the data
    print("\nTrying alternative approach to read the gene data:")
    with gzip.open(matrix_file, 'rt') as file:
        # Find the start of the data
        for line in file:
            if '!series_matrix_table_begin' in line:
                break
                
        # Read the headers and data
        import pandas as pd
        df = pd.read_csv(file, sep='\t', index_col=0)
        print(f"Column names: {df.columns[:5]}")
        print(f"First 20 row IDs: {df.index[:20]}")
        gene_data = df


Found data marker at line 67
Header line: "ID_REF"	"GSM4104672"	"GSM4104673"	"GSM4104674"	"GSM4104675"	"GSM4104676"	"GSM4104677"	"GSM4104678"	"GSM4104679"	"GSM4104680"	"GSM4104681"	"GSM4104682"	"GSM4104683"	"GSM4104684"	"GSM4104685"	"GSM4104686"	"GSM4104687"	"GSM4104688"	"GSM4104689"	"GSM4104690"	"GSM4104691"	"GSM4104692"	"GSM4104693"	"GSM4104694"	"GSM4104695"	"GSM4104696"	"GSM4104697"	"GSM4104698"	"GSM4104699"	"GSM4104700"	"GSM4104701"	"GSM4104702"	"GSM4104703"	"GSM4104704"	"GSM4104705"	"GSM4104706"	"GSM4104707"	"GSM4104708"	"GSM4104709"	"GSM4104710"	"GSM4104711"	"GSM4104712"	"GSM4104713"	"GSM4104714"	"GSM4104715"	"GSM4104716"
First data line: 16650001	1.605655144	1.843828454	1.899724264	1.617480923	1.920638396	3.63385594	1.93408369	1.573835005	1.409345167	2.211837425	1.692594326	1.483653161	1.036818679	1.672944967	3.680559475	2.979731225	3.205596204	3.460458409	1.751848193	0.744824546	3.075519289	2.189577606	1.730044194	2.292415021	2.369373599	2.584867499	3.099427478	1.189063212	1.32

Index(['16650001', '16650003', '16650005', '16650007', '16650009', '16650011',
       '16650013', '16650015', '16650017', '16650019', '16650021', '16650023',
       '16650025', '16650027', '16650029', '16650031', '16650033', '16650035',
       '16650037', '16650041'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Review the gene identifiers in the gene expression data
# The identifiers like '16650001', '16650003', etc. are numeric IDs
# They do not appear to be standard human gene symbols (like BRCA1, TP53, etc.)
# These appear to be probe IDs or other platform-specific identifiers
# They will need to be mapped to standard gene symbols

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['16657436', '16657440', '16657445', '16657447', '16657450'], 'probeset_id': ['16657436', '16657440', '16657445', '16657447', '16657450'], 'seqname': ['chr1', 'chr1', 'chr1', 'chr1', 'chr1'], 'strand': ['+', '+', '+', '+', '+'], 'start': ['12190', '29554', '69091', '160446', '317811'], 'stop': ['13639', '31109', '70008', '161525', '328581'], 'total_probes': [25.0, 28.0, 8.0, 13.0, 36.0], 'gene_assignment': ['NR_046018 // DDX11L1 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 1 // 1p36.33 // 100287102 /// NR_034090 // DDX11L9 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 9 // 15q26.3 // 100288486 /// NR_051985 // DDX11L9 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 9 // 15q26.3 // 100288486 /// NR_045117 // DDX11L10 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 10 // 16p13.3 // 100287029 /// NR_024004 // DDX11L2 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 2 // 2q13 // 84771 /// NR_024005 // DDX11L2 // DEAD/H (Asp-Glu-A

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the columns containing probe IDs and gene symbols
# From gene annotation preview, we can see:
# - 'ID' column contains probe identifiers that match the gene expression data (like '16657436')
# - 'gene_assignment' column contains gene symbols and details (it's quite complex with multiple symbols per probe)

# 2. Create a gene mapping dataframe
# First reusing (still) loaded data from the previous steps
gene_mapping = get_gene_mapping(gene_annotation, 'ID', 'gene_assignment')

# Preview the mapping data to confirm structure
print("Gene mapping preview (first 5 rows):")
print(gene_mapping.head())

# 3. Apply the gene mapping to convert probe level measurements to gene expression data
# We've already acquired the gene expression data in step 3
gene_data = apply_gene_mapping(gene_data, gene_mapping)

# Preview the result
print("\nMapped gene expression data preview (first 5 genes, first 5 samples):")
if not gene_data.empty:
    print(gene_data.iloc[:5, :5])
    print(f"\nShape of gene expression data: {gene_data.shape}")
else:
    print("Warning: Gene expression data is empty after mapping.")


Gene mapping preview (first 5 rows):
         ID                                               Gene
0  16657436  NR_046018 // DDX11L1 // DEAD/H (Asp-Glu-Ala-As...
1  16657440  ENST00000473358 // MIR1302-11 // microRNA 1302...
2  16657445  NM_001005484 // OR4F5 // olfactory receptor, f...
3  16657447                                                ---
4  16657450  AK302511 // LOC100132062 // uncharacterized LO...



Mapped gene expression data preview (first 5 genes, first 5 samples):
      GSM4104672  GSM4104673  GSM4104674  GSM4104675  GSM4104676
Gene                                                            
A-     14.906517   14.453202   14.368576   13.714544   15.999804
A-2     2.291582    2.060694    2.336983    2.143781    2.245854
A-52    3.554224    3.625984    3.578620    3.568357    3.595273
A-E     2.708583    2.514737    2.400064    2.606910    2.643005
A-I     3.774318    3.744129    3.913792    3.843648    3.761335

Shape of gene expression data: (81076, 45)


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Extract clinical features again from the matrix file with proper orientation
# First, get the original clinical data for parsing
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
background_info, clinical_data = get_background_and_clinical_data(matrix_file)

# Define conversion functions from step 2
def convert_trait(value):
    if value is None:
        return None
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    if 'Allogenic FMT' in value:
        return 1  # Allogenic FMT (treatment group)
    elif 'Autologous FMT' in value:
        return 0  # Autologous FMT (control group)
    return None

def convert_age(value):
    if value is None:
        return None
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    try:
        return float(value)  # Convert to numeric value
    except (ValueError, TypeError):
        return None

def convert_gender(value):
    if value is None:
        return None
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    if value == '1':
        return 0  # Female (note: we convert to our standard where female=0, male=1)
    elif value == '0':
        return 1  # Male
    return None

# Extract clinical features with proper orientation
trait_row = 6  # 'experimental condition' in the sample characteristics 
age_row = 3    # 'age (yrs)' in the sample characteristics
gender_row = 1  # 'sex (female=1, male=0)' in the sample characteristics

selected_clinical_df = geo_select_clinical_features(
    clinical_df=clinical_data,
    trait=trait,
    trait_row=trait_row,
    convert_trait=convert_trait,
    age_row=age_row,
    convert_age=convert_age,
    gender_row=gender_row,
    convert_gender=convert_gender
)

# Save clinical data correctly
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
selected_clinical_df.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")

# 3. Link the clinical and genetic data
# First, convert gene data to samples-as-rows format
gene_data_t = normalized_gene_data.T

# Since GEO data often has sample IDs as column names in gene data and as values in the clinical data's first column,
# we need to align them correctly
print("Clinical data columns:", selected_clinical_df.columns.tolist())
print("Clinical data shape:", selected_clinical_df.shape)
print("Gene data shape:", gene_data_t.shape)

# Extract sample IDs from both datasets for comparison
clinical_samples = selected_clinical_df.columns.tolist()
gene_samples = gene_data_t.index.tolist()

# Convert to proper format if needed (removing quotes, etc.)
clinical_samples_clean = [s.strip('"') if isinstance(s, str) else s for s in clinical_samples]
gene_samples_clean = [s.strip('"') if isinstance(s, str) else s for s in gene_samples]

print(f"First few clinical samples: {clinical_samples_clean[:5]}")
print(f"First few gene samples: {gene_samples_clean[:5]}")

# We need to transpose the clinical data to have samples as rows
selected_clinical_df_t = selected_clinical_df.T

# Now link the data - both dataframes should have samples as rows
linked_data = pd.concat([selected_clinical_df_t, gene_data_t], axis=1)
print(f"Linked data shape: {linked_data.shape}")

# 4. Handle missing values in the linked data
linked_data = handle_missing_values(linked_data, trait)
print(f"Data after handling missing values: {linked_data.shape}")

# 5. Determine whether the trait and demographic features are biased
is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data, trait)

# 6. Conduct quality validation and save cohort info
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True, 
    is_trait_available=True, 
    is_biased=is_trait_biased, 
    df=unbiased_linked_data,
    note="Dataset contains gene expression data from IBS patients with different intervention types (allogenic vs autologous FMT)."
)

# 7. Save the linked data if it's usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    unbiased_linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Data was determined to be unusable and was not saved")

Normalized gene data saved to ../../output/preprocess/Irritable_bowel_syndrome_(IBS)/gene_data/GSE138297.csv
Clinical data saved to ../../output/preprocess/Irritable_bowel_syndrome_(IBS)/clinical_data/GSE138297.csv
Clinical data columns: ['GSM4104672', 'GSM4104673', 'GSM4104674', 'GSM4104675', 'GSM4104676', 'GSM4104677', 'GSM4104678', 'GSM4104679', 'GSM4104680', 'GSM4104681', 'GSM4104682', 'GSM4104683', 'GSM4104684', 'GSM4104685', 'GSM4104686', 'GSM4104687', 'GSM4104688', 'GSM4104689', 'GSM4104690', 'GSM4104691', 'GSM4104692', 'GSM4104693', 'GSM4104694', 'GSM4104695', 'GSM4104696', 'GSM4104697', 'GSM4104698', 'GSM4104699', 'GSM4104700', 'GSM4104701', 'GSM4104702', 'GSM4104703', 'GSM4104704', 'GSM4104705', 'GSM4104706', 'GSM4104707', 'GSM4104708', 'GSM4104709', 'GSM4104710', 'GSM4104711', 'GSM4104712', 'GSM4104713', 'GSM4104714', 'GSM4104715', 'GSM4104716']
Clinical data shape: (3, 45)
Gene data shape: (45, 23274)
First few clinical samples: ['GSM4104672', 'GSM4104673', 'GSM4104674', 'G

Data after handling missing values: (45, 23277)
For the feature 'Irritable_bowel_syndrome_(IBS)', the least common label is '0.0' with 21 occurrences. This represents 46.67% of the dataset.
The distribution of the feature 'Irritable_bowel_syndrome_(IBS)' in this dataset is fine.

Quartiles for 'Age':
  25%: 28.0
  50% (Median): 36.0
  75%: 49.0
Min: 21.0
Max: 59.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 21 occurrences. This represents 46.67% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.

A new JSON file was created at: ../../output/preprocess/Irritable_bowel_syndrome_(IBS)/cohort_info.json


Linked data saved to ../../output/preprocess/Irritable_bowel_syndrome_(IBS)/GSE138297.csv
